# Fetch tiles from Landsat

The code in this notebook shows the process of fetching landsat tiles from the USGS server, changing the format of the data, checking for actual data compatibility, and pre-processing into (file,window) pairs.

For the first task, fetching from USGS, you need to have a USGS account, and have the username and password in the USGS_USER and USGS_PASSWORD environment variables, respectively.  

In [5]:
from landsat5fetch import *
from pathlib import Path
import numpy as np
import os

storage_directory = Path(os.getenv('LANDSAT_DIR'))  # set storage_directory however you want

In [6]:
# Prepare session with USGS Server
s = get_session()

In [17]:
# Create list of candidate landsat tiles 
# The file landsat5list.py contains a list of potential tiles that overlap the Corine map, and occur in the same time
# frame as the Corine report.
# (Obviously you could substitute any other tile list you wanted.)

import landsat5list
lst = np.array(landsat5list.potential_tiles)
wehave = list(storage_directory.glob('*.tif'))
candidates = list(set(lst) - set( [x.stem for x in wehave ]))
print( len(lst), len(wehave), len(candidates) )

In [8]:
# Select some tiles at random
# USGS seems to have a limit of 20 that it will allow you to request at a time
np.random.shuffle(candidates)
tofetch = list(candidates[:20])
tofetch

In [11]:
submit_order(list(tofetch))
# Note: if you get a 400 (BAD REQUEST) error, it is probably because you've exceeded some resource limit, 
# not actually a bad request

In [7]:
# Confirm the order
get_open_orders()

In [4]:
# After you have received the 'download ready' email from USGS,
# you can download the results.
downloaded = download_available_results(storage_directory)
len(downloaded)

# Convert the Files
Now we need to process the data from the form we get it from landsat (a gzipped tar file) into the form we  will use (a tif file). Ideally we'd do that processing right here in the notebook, but I haven't figured out how to set a conda environment for subshell in jupyter.  So for now, you have to open a terminal, activate the right conda environment, then run the shell script ls5munge.sh manually.
````
# Usage:  ./ls5munge.sh <landsat_diretory>/*.tar.gz
````
Note: it will automatically skip files that have already been processed, so you don't have to worry about telling it exactly which ones are the new ones.

# Analyze Tiles
In this section, we analyze the downloaded results for compatibility with the Corine data.

In [12]:
# append the parent directory to the python path so that we can import tools for analysis
import sys
sys.path.append(str(Path("..").resolve()))

from multispectral import corine
from multispectral import coords
from multispectral import tools
import rasterio
tools.set_figure_width(15)

In [13]:
#tocheck = list(storage_directory.glob('*.tif'))  # to look at all files
tocheck = [ storage_directory / (x + ".tif") for x in downloaded ]
len(tocheck)

Not all the tiles in landsat5list.py will actually be useful--they won't all intersect the Corine data.
There are two reasons for this: (1) The tiles in landsat5list.py were generated from an _approximation_ of the 
Corine outline, and (2) both the tiles and the Corine data don't completely fill their rectangular extents, so even when the rectangles overlap, the data may not.

The loop below will find tiles that don't have a proper overlap.
For any file identified here, you should remove it from storage_directory, *and* comment it out of landsat5list.py to
prevent it from being downloaded again. In fact, you can comment out the entire pathrow combination (see the file for examples).

In [14]:
def smudge(pw1, pw2):
    """Due to rounding errors, it is possible that the same geo window results in pixel windows of two different sizes
    for different data files.  Smudge adjusts a matching pair of pixel windows so that they are definitely the same
    size.   Currently nothing clever here about geo registration; just making the height/width match."""
    common_height = min(pw1.height,pw2.height)
    common_width = min(pw1.width,pw2.width)
    if pw1.width != common_width or pw1.height != common_height:
        pw1 = rasterio.windows.Window(pw1.col_off,pw1.row_off,common_width,common_height)
    if pw2.width != common_width or pw2.height != common_height:
        pw2 = rasterio.windows.Window(pw2.col_off,pw2.row_off,common_width,common_height)
    return (pw1,pw2)


failed = []
successful = []
for file in tocheck:
    fp = rasterio.open(file)
    cp = corine.fetch_corine(fp.crs)
    geo_common = coords.geo_window_intersect(fp.bounds,cp.bounds)
    if geo_common is None:
        # This case shouldn't happen, and probably should be investigated.
        failed.append(file)
        print("{} does not intersect {}".format(file.name, cp.name))
        continue
    
    fp_window = coords.geo_to_pixel(fp,geo_common)
    cp_window = coords.geo_to_pixel(cp,geo_common)
    (fp_window,cp_window) = smudge(fp_window,cp_window)
    
    # check to see if the non-nodata bits overlap
    fp_patch = fp.read(7, window=fp_window)
    if not fp_patch.any():
        failed.append(file)
        print("{} is empty in intersection".format(file.name))
        continue
        
    cp_patch = cp.read(1, window=cp_window)
    if not cp_patch.any():
        failed.append(file)
        print("{} intersects empty part of corine".format(file.name))
        continue
    
    common_data = np.logical_and(fp_patch,cp_patch)
    if not common_data.any():
        failed.append(file)
        print("{} and {} have no common data".format(file.name, cp.name))
        continue
    
    successful.append(file)

In [15]:
len(failed), len(successful)

Show the location of tiles superimposed on the corine maps.  You should generally expect failed tiles to be visibly outside or on the edge 
of the corine data.  If they are not, it may mean the landsat data is corrupted, or a bug, or...

(Note that some of the Corine outlines may be a bit hard to recognize because the corine data extends out into the ocean for some ways.  But rest assured each segment is some slice of Europe.)

In [ ]:
for file in failed:  # 'tocheck', 'failed' or 'successful'
    fp = rasterio.open(file)
    cp = corine.fetch_corine(fp.crs)
    tools.show_bands(cp,1,showrect=coords.geo_to_pixel(cp,fp.bounds).flatten())

# Precompute Windows
The data is now available for use.  However for efficiency we usually preprocess it to create a master list of (tile, window) tuples.

In [ ]:
# If you didn't do this above, do it now
# append the parent directory to the python path so that we can import tools for analysis
import sys
sys.path.append(str(Path("..").resolve()))

from multispectral import corine
from multispectral import coords
from multispectral import tools
import rasterio
tools.set_figure_width(15)

In [ ]:
# The file where we will store the master list of all windows.
window_list_file = storage_directory / "all_windows.csv"

In [ ]:
# to_process = list(storage_directory.glob('*.tif'))  # If you want to recompute all windows from scratch:
to_process = successful  # to add new files to the list
to_process[:5]

Presumably we are only processing tiles that have valid overlap with the Corine data.  But even though the tile may have overlap, individual windows may not.  In fact, many windows around the edge of the tile will have only NODATA values, and so not be useful to us at all.  Prefiltering filters out the NODATA windows and also does the same kind of overlap analysis we did above with the Corine data.  The windows that are left are known to have good overlapping data. 

In [ ]:
windows_list = list( windows.prefilter(to_process, corine.corine_filter)) 
len(windows_list)

In [ ]:
# As an extra check, you can see how many windows of each type were produced by each tile.
corine._tile_counts

Store the windows for re-use.  I'm keeping all windows together in one master file that is used as input to the learners.  Store them however you like

In [ ]:
# Add the new list to the master list
windows.to_file(windows_list, window_list_file, append=True)

In [ ]:
# or, to read from the file
# windows_list = windows.from_file(windows_list_file)

It is nice to see what you actually have. This code will show the windows superimposed on the Corine map segments.  I find it very gratifying to see the outlines of the Corine data neatly mapped to windows. :-)
You can do this even with many thousands of windows, but not surprisingly it can get kind of slow.

In [ ]:
# First sort the windows by corine segment
cdats = {}
for (ld,w) in windows_list:
    cdat = corine.fetch_corine(ld.crs)
    if cdat not in cdats:
        cdats[cdat] = []
    cdats[cdat].append((ld,w))

# How many windows on each segment?
for cdat in cdats:
    print("{}: {}".format(cdat.name, len(cdats[cdat])))

In [ ]:
# show them all...
for cdat, wl in cdats.items():
    rs = [coords.geo_to_pixel(cdat, coords.pixel_to_geo(x,y)).flatten() for (x,y) in wl ]
    tools.show_band(cdat,1,showrect=rs)